Сначала создадим оптическую систему:

In [ ]:
# type: ignore
from pint import UnitRegistry  # спрячем это в библиотеку
ureg = UnitRegistry()  # спрячем это в библиотеку


simulation_params = SimulationParams(
    Nx=1000,
    Ny=1000,
    lx=5 * ureg.mm,
    ly=10 * ureg.mm,
    wavelenght = 500 * ureg.nm
)


beam = GaussBeam(# пока только в центре
    x = 0 * ureg.mm,
    y = 0 * ureg.mm,
    dx = 1 * ureg.mm,
    dy = 1 * ureg.mm,
)

lens = ThinLens(
    F = 1/10  # в расстояниях
)

free_space = FreeSpace(
    d = 10 * ureg.mm,
    method = FreeSpace.FRAUNHOFER  # нужно подумать, как лучше реализовать, чтобы потом была возможность расширять
)

slm = SLM(
    mask = np.zeros(...)
)

initial_system = LinearSystem(
    [
        lens,
        free_space,
        slm
    ],
    simulation_params
)

### Кейс 1 ###

После инициализации оптической системы можно решить две задачи оптимизации: генерация заданного распределения интенсивности и восстановление фазы волнового фронта. 

Для первой задачи основными параметрами являются два профиля интенсивности: преобразуемый профиль(source) и искомый профиль(target)

Для второй задачи основными параметрами являются два профиля интенсивности и фазовое распределение: преобразуемый профиль(source),искомый профиль(target) и фазовое распределение(phase)

Случай высокоуровнего API: прямое и обратное распространение волнового фронта рассчитывается на основании методов forward, уже реализованных внутри классов из модуля elements. Соответственно должен быть реализованы методы forward и reverse у класса LinearSystem, один из которых генерирует волновой фронт, падающий на детектор, а другой генерирует волновой фронт падающий на источник

In [ ]:
intensity_target = torch.tensor(...)
intensity_source = torch.tensor(...)

# для задачи генерации требуемого профиля
phase_mask = optimize.goal_target(
    target=intensity_target,
    source=intensity_source,
    setup=initial_system,
    tol=1e-5,
    method='HIO'
)

# для задачи восстановления фазы
phase_profile = torch.tensor([......])

phase = optimize.goal_phase(
    phase=phase_profile,
    target=intensity_target,
    source=intensity_source,
    setup=initial_system,
    tol=1e-5,
    method='HIO'
)


Случай низкоуровнего API: необходимо задать функцияя прямого и обратного распространения волнового фронта через оптическую систему

In [ ]:
intensity_target = torch.tensor(...)
intensity_source = torch.tensor(...)

def forward(field_input):
    ....
    return field_output

def reverse(field_output):
    ...
    return field_input



# для задачи генерации требуемого профиля
phase_mask = optimize.goal_target(
    target=intensity_target,
    source=intensity_source,
    setup=None,
    forward_propagation=forward,
    reverse_propagation=reverse,
    tol=1e-5,
    method='GS'
)

# для задачи восстановления фазы
phase_profile = torch.tensor([......])

phase = optimize.goal_phase(
    phase=phase_profile,
    target=intensity_target,
    source=intensity_source,
    setup=None,
    forward_propagation=forward,
    reverse_propagation=reverse,
    tol=1e-5,
    method='GS'
)

### Кейс 2 ###
Здесь попытка в ООП: создаем общий класс OptimizeProblem, который инициализируется двумя заданными профилями, точностью и алгоритмом оптимизации. Что-то такое:
```python
class OptimizeProblem:
    def __init__(
        self, 
        target_intensity, 
        source_intensity,
        optical_setup, 
        simulation_parameters: Simulation_Parameters
        tol: float 1e-5
    ):
    
От него наследуем два класса, характеризующих 2 задачи: генерация заданного профиля и восстановление волнового фронта:

```python
class PhaseRetrievalProblem(OptimizeProblem):
    def __init__(self, phase_profile):

    super.__init__(OptimizeProblem)
    ...

class TargetProblem(OptimizeProblem):
    def __init__(self, mask_format: Literal['grey', '2pi'])
    ...


Случай высокоуровнего API:

In [ ]:
intensity_target = torch.tensor(...)
intensity_source = torch.tensor(...)

# для задачи генерации требуемого профиля
params = SimulationParameters(...)
task_intensity = optimize.TargetProblem(
    target_intensity=intensity_target,
    source_intensity=intensity_source,
    optical_setup=initial_system...
)


# для задачи восстановления фазы
phase_profile = torch.tensor([......])
task_phase = optimize.PhaseRetrievalProblem(
    target_intensity=intensity_target,
    source_intensity=intensity_source,
    optical_setup=initial_system...
    phase_profile=phase_profile
)

# Дальше вызываем метод оптимизации с указанным алгоритмом

mask = task_intensity.optimize(method='HIO')
profile = task_phase.optimize(method='HIO')



Низкоуровневый API: указываем собственные функции распространения и вызываем другой метод

In [ ]:
def forward(field_input):
    ....
    return field_output

def reverse(field_output):
    ...
    return field_input

In [ ]:
mask = task_intensity.optimize_lowapi(forward=forward, reverse=reverse, method='HIO')
profile = task_phase.optimizelowapi(forward=forward, reverse=reverse, method='HIO')